# 项目简介
安全帽佩戴检测是计算机视觉在工业安全领域应用的典型场景，本文使用PaddleX动态图进行yolov3_mobilenet迁移学习训练。PaddleX升级到2.0.0-rc，基于动态图的训练和预测依然非常便捷。

## 前置项目：基于Paddle 1.8.4环境的安全帽检测模型训练与部署合集
- [PaddleX、PP-Yolo：手把手教你训练、加密、部署目标检测模型](https://aistudio.baidu.com/aistudio/projectdetail/920753)
	- 项目简介：基于PaddleX的pp-yolo迁移学习，通过检测人员是否戴安全帽来提高工作安全度，加密模型并使用C++本地部署，同时提供HubServing在线部署实现。
- [安全帽佩戴检测模型训练与一键部署（PaddleX、HubServing）](https://aistudio.baidu.com/aistudio/projectdetail/742090)
	- 项目简介：基于PaddleX的yolov3\_darknet53迁移学习，通过检测人员是否戴安全帽来提高工作安全度。使用HubServing进行轻量级服务化部署。
- [PaddleLite树莓派从0到1：安全帽检测小车部署（一）](https://aistudio.baidu.com/aistudio/projectdetail/1059610)
	- 项目简介：介绍了在树莓派3B上安装Raspberry Pi OS 64位操作系统、组装树莓派小车、基于PaddleDetection+PaddleLite完成使用ssd\_mobilenet\_v1进行迁移学习的安全帽检测模型部署。
- [巡检告警机器人上线！PaddleLite安全帽检测小车部署（二）](https://aistudio.baidu.com/aistudio/projectdetail/1209733)
	- 项目简介：介绍在树莓派3B上实现web端实时监控、控制LED灯与蜂鸣器报警的代码实现（C++、Python、C）。同时，将基于PaddleLite的安全帽检测模型与蜂鸣器报警结合，出现未佩戴安全帽的目标时，蜂鸣器会产生报警，并保存目标画面。
- [部署PPDET训练的YoloV3模型：PaddleLite安全帽检测小车（三）](https://aistudio.baidu.com/aistudio/projectdetail/1292497)
	- 项目简介：介绍树莓派上基于PaddleDetection训练的yolov3安全帽检测部署、蜂鸣器报警功能的实现。
- [YoloV3检测模型在树莓派上的部署（PaddleX、PaddleLite）](https://aistudio.baidu.com/aistudio/projectdetail/1227445)
	- 项目简介：在树莓派上部署基于PaddleX训练的yolov3\_mobilenetv3模型。

# 准备模型库
在新发布的2.0.0-rc版本中，PaddleX全面升级了对动态图的支持。

项目环境：Paddle 2.1.0

参考文档：[https://gitee.com/paddlepaddle/PaddleX/tree/release%2F2.0-rc](https://gitee.com/paddlepaddle/PaddleX/tree/release%2F2.0-rc)

In [ ]:
!pip install paddlex==2.0rc

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 450kB 15.4MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 14.3MB/s eta 0:00:01
     |████████████████████████████████| 307kB 17.1MB/s eta 0:00:01


到这里，我们就成功安装了PaddleX的2.0.0-rc版本，请注意，在Paddle 2.0以上的环境中，不能使用PaddleX 1.X的版本，否则后续在数据集切分时就会报错。

# 数据集准备
本项目使用的安全帽检测数据集已经按VOC格式进行标注，目录情况如下：
```
 dataset/                        
 ├── annotations/    
 ├── images/       
 ```
而使用PaddleX的API，一键进行数据切分时，数据文件夹切分前后的状态如下:
```
 dataset/                          dataset/
 ├── Annotations/      <--         ├── Annotations/
 ├── JPEGImages/                   ├── JPEGImages/
 ├── labels.txt
 ├── test_list.txt
 ├── train_list.txt
 ├── val_list.txt
 ```
 使用稍加整理，就可以使用一键切分快速划分数据集。

In [ ]:
# 解压数据集，并做好数据切分准备

In [ ]:
!unzip /home/aistudio/data/data50329/HelmetDetection.zip -d ./MyDataset

In [ ]:
!mv MyDataset/images MyDataset/JPEGImages

In [ ]:
!mv MyDataset/annotations MyDataset/Annotations

将训练集、验证集和测试集按照7：2：1的比例划分。 PaddleX中提供了简单易用的API，方便用户直接使用进行数据划分。下面这行代码正确执行的前提是，PaddleX的版本和Paddle匹配，要么都是2.0+，要么都是1.8.X。

In [ ]:
!paddlex --split_dataset --format VOC --dataset_dir MyDataset --val_value 0.2 --test_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."
[08-12 14:43:24 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --split_dataset --format VOC --dataset_dir MyDataset --val_value 0.2 --test_value 0.1
[08-12 14:43:24 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_seriali

到这里，基于PaddleX强大的API功能，安全帽检测数据集也完成了整理、划分。

# 模型训练
PaddleX最核心的特点就是模型训练极其简单，比如下面几行代码演示的，只要配置了数据集目录，选择好模型即可。

In [ ]:
import paddlex as pdx
from paddlex import transforms as T

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[08-12 14:43:43 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'floating': np.float,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of

In [ ]:
# 定义训练和验证时的transforms
# API说明：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0-rc/paddlex/cv/transforms/operators.py
train_transforms = T.Compose([
    T.MixupImage(mixup_epoch=250), T.RandomDistort(),
    T.RandomExpand(im_padding_value=[123.675, 116.28, 103.53]), T.RandomCrop(),
    T.RandomHorizontalFlip(), T.BatchRandomResize(
        target_sizes=[320, 352, 384, 416, 448, 480, 512, 544, 576, 608],
        interp='RANDOM'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

eval_transforms = T.Compose([
    T.Resize(
        608, interp='CUBIC'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# 定义训练和验证所用的数据集
# API说明：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0-rc/paddlex/cv/datasets/voc.py#L29
train_dataset = pdx.datasets.VOCDetection(
    data_dir='MyDataset',
    file_list='MyDataset/train_list.txt',
    label_list='MyDataset/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.VOCDetection(
    data_dir='MyDataset',
    file_list='MyDataset/val_list.txt',
    label_list='MyDataset/labels.txt',
    transforms=eval_transforms,
    shuffle=False)

2021-08-12 14:43:56 [INFO]	Starting to read file list from dataset...
2021-08-12 14:44:05 [INFO]	3500 samples in file MyDataset/train_list.txt
creating index...
index created!
2021-08-12 14:44:05 [INFO]	Starting to read file list from dataset...
2021-08-12 14:44:08 [INFO]	1000 samples in file MyDataset/val_list.txt
creating index...
index created!


涉及到PaddleX中目标检测模型的选择，最关键的就是这行代码

`pdx.models.YOLOv3(num_classes=num_classes, backbone='DarkNet53')`

可以先查看下PaddleX目前提供了哪些目标检测模型以及`backbone`的支持。

In [ ]:
??pdx.models.FasterRCNN

In [ ]:
??pdx.models.YOLOv3

In [ ]:
# 初始化模型，并进行训练
# 可使用VisualDL查看训练指标，参考https://github.com/PaddlePaddle/PaddleX/tree/release/2.0-rc/tutorials/train#visualdl可视化训练指标
num_classes = len(train_dataset.labels)
model = pdx.models.YOLOv3(num_classes=num_classes, backbone='MobileNetV3_ssld')

In [17]:
# API说明：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0-rc/paddlex/cv/models/detector.py#L155
# 各参数介绍与调整说明：https://paddlex.readthedocs.io/zh_CN/develop/appendix/parameters.html
model.train(
    num_epochs=20,
    train_dataset=train_dataset,
    train_batch_size=2,
    eval_dataset=eval_dataset,
    learning_rate=0.001 / 8,
    warmup_steps=1000,
    warmup_start_lr=0.0,
    save_interval_epochs=1,
    lr_decay_epochs=[216, 243],
    save_dir='output/yolov3_mobilenet')

可视化训练过程：
![](https://ai-studio-static-online.cdn.bcebos.com/a4242c67a6f24c9294c56f8187db000c9748fb7604cc49dca238522a3dc5a1a7)


# 模型预测
这里，我们不仅可以挑选测试集图片进行预测，还可以写一个简单的逻辑，完成图片中目标数量的计数。比如，我们可以统计下一张图片中，共有多少人未佩戴安全帽。

In [18]:
??pdx.visualize_detection

In [19]:
import glob
import numpy as np
import threading
import time
import random
import os
import base64
import cv2
import json
import paddlex as pdx
# 传入待预测图片
image_name = 'MyDataset/JPEGImages/hard_hat_workers1035.png'
# 模型保存位置
model = pdx.load_model('output/yolov3_mobilenet/best_model')

img = cv2.imread(image_name)
result = model.predict(img)

keep_results = []
areas = []
f = open('./output/yolov3_mobilenet/result.txt', 'a')
count = 0
for dt in np.array(result):
    cname, bbox, score = dt['category'], dt['bbox'], dt['score']
    if score < 0.5:
        continue
    keep_results.append(dt)
    # 检测到未佩戴安全帽的目标，计数加1
    if cname == 'head': 
        count += 1
    f.write(str(dt) + '\n')
    f.write('\n')
    areas.append(bbox[2] * bbox[3])
areas = np.asarray(areas)
sorted_idxs = np.argsort(-areas).tolist()
keep_results = [keep_results[k]
                for k in sorted_idxs] if len(keep_results) > 0 else []
print(keep_results)
print(count)
f.write("未佩戴安全帽人员总数为: " + str(int(count)))
f.close()

2021-08-12 17:20:39 [INFO]	Model[YOLOv3] loaded.
[{'category_id': 0, 'category': 'head', 'bbox': [207.8216552734375, 188.14649963378906, 20.7681884765625, 24.755584716796875], 'score': 0.7588264346122742}, {'category_id': 0, 'category': 'head', 'bbox': [174.79931640625, 188.2290496826172, 20.795257568359375, 24.660491943359375], 'score': 0.8966265320777893}, {'category_id': 0, 'category': 'head', 'bbox': [59.40483856201172, 186.1151580810547, 20.54144287109375, 24.113311767578125], 'score': 0.9385504722595215}, {'category_id': 0, 'category': 'head', 'bbox': [135.0240478515625, 189.11782836914062, 20.638275146484375, 23.9400634765625], 'score': 0.8429598808288574}, {'category_id': 0, 'category': 'head', 'bbox': [300.1785583496094, 186.74708557128906, 20.674560546875, 22.866119384765625], 'score': 0.9601511359214783}, {'category_id': 0, 'category': 'head', 'bbox': [19.802230834960938, 183.97540283203125, 20.374671936035156, 22.974609375], 'score': 0.8881982564926147}, {'category_id': 0, 

In [20]:
pdx.visualize_detection(
    image_name, result, threshold=0.5, save_dir='./output/yolov3_mobilenet')

2021-08-12 17:20:49 [INFO]	The visualized result is saved at ./output/yolov3_mobilenet/visualize_hard_hat_workers1035.png


预测结果：
```
{'category_id': 0, 'category': 'head', 'bbox': [301.3816223144531, 187.12403869628906, 18.0198974609375, 23.79364013671875], 'score': 0.9641125202178955}

{'category_id': 0, 'category': 'head', 'bbox': [98.42637634277344, 186.95138549804688, 17.62054443359375, 22.250213623046875], 'score': 0.9461163282394409}

{'category_id': 0, 'category': 'head', 'bbox': [60.45779037475586, 186.9356231689453, 18.03213119506836, 24.173797607421875], 'score': 0.940744161605835}

{'category_id': 0, 'category': 'head', 'bbox': [244.22019958496094, 187.01390075683594, 16.083084106445312, 22.99420166015625], 'score': 0.940227210521698}

{'category_id': 0, 'category': 'head', 'bbox': [136.5784912109375, 189.411865234375, 18.008148193359375, 22.964599609375], 'score': 0.8962541222572327}

{'category_id': 0, 'category': 'head', 'bbox': [21.016231536865234, 183.6320343017578, 17.833927154541016, 23.470489501953125], 'score': 0.886038601398468}

{'category_id': 0, 'category': 'head', 'bbox': [176.54640197753906, 188.69781494140625, 17.38409423828125, 23.967681884765625], 'score': 0.8675879240036011}

{'category_id': 0, 'category': 'head', 'bbox': [337.97821044921875, 186.59945678710938, 16.95550537109375, 22.102203369140625], 'score': 0.8598908185958862}

{'category_id': 0, 'category': 'head', 'bbox': [267.6236267089844, 185.5501708984375, 16.89678955078125, 23.020751953125], 'score': 0.8243097066879272}

{'category_id': 0, 'category': 'head', 'bbox': [210.44265747070312, 189.28919982910156, 18.027557373046875, 23.715240478515625], 'score': 0.786725640296936}

{'category_id': 0, 'category': 'head', 'bbox': [376.5035400390625, 195.9794158935547, 14.513916015625, 20.0159912109375], 'score': 0.5946234464645386}

未佩戴安全帽人员总数为: 11
```

![](https://ai-studio-static-online.cdn.bcebos.com/bbe78d37d28f4b49882a870bc989b0400d2c6744f7b04f03a80264ced9ad6286)


# 模型裁剪

模型裁剪可以更好地满足在端侧、移动端上部署场景下的性能需求，可以有效得降低模型的体积，以及计算量，加速预测性能。PaddleX集成了PaddleSlim的基于敏感度的通道裁剪算法，通过以下代码，可以在此前训练模型的基础上，加载并进行裁剪，重新开始训练。

In [ ]:
# 加载模型
model = pdx.load_model('output/yolov3_mobilenet/best_model')

# Step 1/3: 分析模型各层参数在不同的剪裁比例下的敏感度
# API说明：https://github.com/PaddlePaddle/PaddleX/blob/95c53dec89ab0f3769330fa445c6d9213986ca5f/paddlex/cv/models/base.py#L352
model.analyze_sensitivity(
    dataset=eval_dataset,
    batch_size=1,
    save_dir='output/yolov3_mobilenet/prune')

# Step 2/3: 根据选择的FLOPs减小比例对模型进行剪裁
# API说明：https://github.com/PaddlePaddle/PaddleX/blob/95c53dec89ab0f3769330fa445c6d9213986ca5f/paddlex/cv/models/base.py#L394
model.prune(pruned_flops=.2)

# Step 3/3: 对剪裁后的模型重新训练
# API说明：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0-rc/paddlex/cv/models/detector.py#L154
# 各参数介绍与调整说明：https://paddlex.readthedocs.io/zh_CN/develop/appendix/parameters.html
model.train(
    num_epochs=270,
    train_dataset=train_dataset,
    train_batch_size=8,
    eval_dataset=eval_dataset,
    learning_rate=0.001 / 8,
    warmup_steps=1000,
    warmup_start_lr=0.0,
    save_interval_epochs=5,
    lr_decay_epochs=[216, 243],
    pretrain_weights=None,
    save_dir='output/yolov3_mobilenet/prune')

# 小结
全面支持动态图的PaddleX，操作流程依然非常简单，在16个小时内，可以很快地完成一个效果良好的安全帽检测模型训练。后续，将继续探索模型在边缘端设备如EdgeBoard的部署实现。